<a href="https://colab.research.google.com/github/imsewonyyy/AI-/blob/main/A%EB%AC%B8%EC%A0%9C_%EB%AA%A8%EB%8D%B8%EB%A7%81_%EB%8D%B0%EC%9D%B4%ED%84%B0%ED%83%90%EC%83%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd

from google. colab import files
uploaded=files.upload()

Saving Airfares_simple_garbled.csv to Airfares_simple_garbled.csv


In [30]:
from google. colab import files
uploaded=files.upload()

Saving Airfares.csv to Airfares.csv


In [10]:
# 수정된 데이터셋 변수
df = pd.read_csv("Airfares_simple_garbled.csv")

In [31]:
# 원본 데이터셋 변수
df_orig = pd.read_csv('Airfares.csv')

In [12]:
# 1. 결측치 찾기
missing_counts = df.isnull().sum()
missing_cols = missing_counts[missing_counts > 0]
print("=== 1. 결측치 (Missing Values) ===")
print(missing_cols, "\n")

=== 1. 결측치 (Missing Values) ===
S_INCOME    32
E_INCOME    33
DISTANCE    31
PAX         31
dtype: int64 



In [14]:
# 2. 문자열 공백(Whitespace) 찾기
print("=== 2. 문자열 공백(Whitespace) 문제 ===")
whitespace_issues = {}                                    # 공백 발생한 값 담을 딕셔너리
for col in df.select_dtypes(include='object').columns:                       # 문자열만을 골라내서
    mask = df[col].apply(lambda x: isinstance(x, str) and (x != x.strip()))  # x != x.strip() / 실제값 x가 양쪽 공백을 제거한 x와 다르다면,
    if mask.any():                                                           # 공백이 존재한다는 의미. => 문자열 공백 이슈가 발생한 값
        whitespace_issues[col] = df.loc[mask, col]
        print(f"\nColumn: {col}")
        print(df.loc[mask, col])

=== 2. 문자열 공백(Whitespace) 문제 ===

Column: E_CITY
36       Charlotte           NC  
51       Cleveland           OH  
101      Hartford            CT  
103      Honolulu (Intl)     HI  
113      Houston             TX  
178      Miami               FL  
195      Minneapolis/St Paul MN  
235      New York/Newark     NY  
243      New York/Newark     NY  
275      New York/Newark     NY  
360      Phoenix             AZ  
443      Salt Lake City      UT  
487      San Francisco       CA  
499      San Jose            CA  
568      Tucson              AZ  
Name: E_CITY, dtype: object


In [25]:
# 앞 뒤로 공백있는 문자열 공백값 존재 예시 출력

df['E_CITY'][36]

'  Charlotte           NC  '

In [26]:
# 3. 대소문자 불일치(Case Inconsistency) 찾기
print("\n=== 3. 대소문자 불일치(Case Inconsistency) ===")
case_issues = {}
for col in df.select_dtypes(include='object').columns:   # 문자열 타입인 모든 컬럼의 값 가져와서 (결측치, 중복값제거, 문자열str만)
    vals = df[col].dropna().astype(str)
    lower_vals = vals.str.lower()                        # 소문자로 모두 변환한 시리즈(base) 만들고
    unique_lower = lower_vals.unique()                   # 실제값(variants)이랑 비교해가며 일치하는 값은 '대소문자 불일치' 값 !
    for base in unique_lower:                                    # 일치한다는 것은 실제값이 소문자로 되어있다는 의미이기에.
        variants = sorted(vals[lower_vals == base].unique())
        if len(variants) > 1:
            if col not in case_issues:
                case_issues[col] = {}
            case_issues[col][base] = variants
    if col in case_issues:
        print(f"\nColumn: {col}")
        for base, variants in case_issues[col].items():
            print(f"  Base ('{base}') -> Variants: {variants}")


=== 3. 대소문자 불일치(Case Inconsistency) ===

Column: S_CITY
  Base ('atlanta             ga') -> Variants: ['Atlanta             GA', 'atlanta             ga']
  Base ('boston              ma') -> Variants: ['Boston              MA', 'boston              ma']
  Base ('chicago             il') -> Variants: ['Chicago             IL', 'chicago             il']
  Base ('fort lauderdale     fl') -> Variants: ['Fort Lauderdale     FL', 'fort lauderdale     fl']
  Base ('los angeles         ca') -> Variants: ['Los Angeles         CA', 'los angeles         ca']
  Base ('new orleans         la') -> Variants: ['New Orleans         LA', 'new orleans         la']
  Base ('new york/newark     ny') -> Variants: ['New York/Newark     NY', 'new york/newark     ny']
  Base ('orlando             fl') -> Variants: ['Orlando             FL', 'orlando             fl']
  Base ('phoenix             az') -> Variants: ['Phoenix             AZ', 'phoenix             az']
  Base ('san francisco       ca') -> Varian

In [34]:
# 4. 수치형 이상치(Outlier) 찾기 (원본 IQR 기준)

print("\n=== 4. 수치형 이상치 (Outliers) ===")
outlier_indices = {}
for col in df.select_dtypes(include=['int64','float64']).columns:      # 수정된 데이터셋에서 정수, 실수 타입 데이터만 추
    o_col = df_orig[col]                         # '원본 데이터!!'의 1사분위(하위 25% 지점)와 3사분위(상위 25% 지점)을 구함
    Q1 = o_col.quantile(0.25)
    Q3 = o_col.quantile(0.75)                     #  IQR = Q3 – Q1은 “중간 50% 자료가 퍼져 있는 범위” => 이상치 아닌 구간
    IQR = Q3 - Q1                                 #  일반적으로 QR의 1.5배를 경계로 삼아서
    lower = Q1 - 1.5 * IQR                          # 하한(lower fence) = Q1 − 1.5×IQR
    upper = Q3 + 1.5 * IQR                          # 상한(upper fence) = Q3 + 1.5×IQR 을 벗어난 값을 '이상치'로 간주 !
    g_col = pd.to_numeric(df[col], errors='coerce')
    mask = (g_col < lower) | (g_col > upper)
    if mask.any():                                    # '수정된 데이터셋 값'이 '원본 데이터의 상한, 하한'을 벗어난다면 이상치로 간주 !
        outlier_indices[col] = df.index[mask].tolist()
        print(f"\nColumn: {col} - {len(outlier_indices[col])}개 이상치 발견")
        print(df.loc[outlier_indices[col][:5], [col]].to_string(index=True))


=== 4. 수치형 이상치 (Outliers) ===

Column: COUPON - 23개 이상치 발견
     COUPON
18     1.86
19     1.93
20     1.87
129    1.71
166    1.92

Column: NEW - 67개 이상치 발견
    NEW
9     2
11    1
31    0
33    1
46    1

Column: HI - 14개 이상치 발견
          HI
2    9185.28
50   9350.13
148  9592.99
197  9249.13
324  9649.01

Column: S_INCOME - 9개 이상치 발견
     S_INCOME
21    38813.0
105   38813.0
129   38813.0
349   14600.0
387   38813.0

Column: DISTANCE - 9개 이상치 발견
     DISTANCE
103    2764.0
106    2679.0
162    2605.0
318   13080.0
376    2603.0

Column: PAX - 57개 이상치 발견
         PAX
42   30877.0
43   30877.0
82   34113.0
83   34113.0
104  32824.0

Column: FARE - 10개 이상치 발견
       FARE
36  -234.15
170 -193.50
346 -229.84
354 -195.91
474  367.72


In [38]:
# 5. 음수 값(Negative Values) 찾기  => 범위오류 (불가능한 값)

print("\n=== 5. 음수 값 (Negative Values) ===")
negative_indices = {}
for col in df.select_dtypes(include=['int64','float64']).columns:       # 정수, 실수형 데이터 값 중에
    g_col = pd.to_numeric(df[col], errors='coerce')                # 문자열 등이 섞여 있을 경우에도 가능한 한 숫자로 바꿔주고, 변환이 불가능한 항목은 NaN으로 처리
    neg_mask = g_col < 0                                      # 해당 열의 값이 음수인 행만 true인 시리즈
    if neg_mask.any():                                             # 음수 값이 하나라도 있으면 사전에 저장, 해당 행의 인덱스와 샘플 출력
        negative_indices[col] = df.index[neg_mask].tolist()
        print(f"\nColumn: {col} - {len(negative_indices[col])}개 음수 값 발견")
        print(df.loc[negative_indices[col][:5], [col]].to_string(index=True))


=== 5. 음수 값 (Negative Values) ===

Column: FARE - 5개 음수 값 발견
       FARE
36  -234.15
170 -193.50
346 -229.84
354 -195.91
562 -123.89
